In [1]:
import sqlite3
import csv

In [ ]:
def load_safety_data(data):
    try:
        conn = sqlite3.connect("safety_data.db")
        cursor = conn.cursor()
        cursor.execute("""
         CREATE TABLE IF NOT EXISTS safety (
            employee_id PRIMARY KEY,
            employee_name TEXT,
            department TEXT,
            incident_date Date,
            incident_type TEXT,
            days_lost INT,
            injury_severity TEXT)

            """
        )
        with open(data, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            # skip header
            next(csv_reader)
            for row in csv_reader:
                cursor.execute("INSERT INTO safety VALUES (?, ?, ?, ?, ?, ?, ?)", row)
        conn.commit()
        print("Data loaded successfully")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.close()

data = "/content/company_safety.csv"
load_safety_data(data)

Data loaded successfully


In [3]:
# Loading the extension
%load_ext sql

In [4]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [5]:
# Using the extension to connect to database
%sql sqlite:///safety_data.db

In [6]:
%%sql
SELECT * FROM safety
LIMIT 5;

 * sqlite:///safety_data.db
Done.


employee_id,employee_name,department,incident_date,incident_type,days_lost,injury_severity
1,John Doe,Manufacturing,2023-01-05,Slip,3,Minor
2,Jane Smith,Maintenance,,Fall,5,Moderate
3,Jim Brown,Manufacturing,2023-02-12,Slip,1,Minor
4,Mary Johnson,Quality Control,2023-03-15,,2,Minor
5,Robert Wilson,Maintenance,2023-04-22,Machine Accident,10,Severe


#### 1.  write a query to return how many columns and rows are in the dataset.

In [7]:
%%sql
PRAGMA table_info(safety);

 * sqlite:///safety_data.db
Done.


cid,name,type,notnull,dflt_value,pk
0,employee_id,,0,None,1
1,employee_name,TEXT,0,None,0
2,department,TEXT,0,None,0
3,incident_date,Date,0,None,0
4,incident_type,TEXT,0,None,0
5,days_lost,INT,0,None,0
6,injury_severity,TEXT,0,None,0


In [8]:
%%sql
SELECT COUNT(*) AS number_of_rows
FROM safety;

 * sqlite:///safety_data.db
Done.


number_of_rows
30


In [9]:
%%sql
SELECT COUNT(*) AS number_of_rows
FROM safety
WHERE days_lost is NULL;

 * sqlite:///safety_data.db
Done.


number_of_rows
0


#### 2. How many rows in the incident_date column has missing values or empty strings? Write a query to return the total number of missing values in this column.

In [10]:
%%sql
SELECT COUNT(*) AS missing_empty_strings_rows
FROM safety
WHERE incident_date IS NULL OR incident_date = '';

 * sqlite:///safety_data.db
Done.


missing_empty_strings_rows
3


#### 3. Write a query to return all the rows in the table where "incident_date" OR  "days_lost" values are missing or null.

In [11]:
%%sql
SELECT * FROM safety
WHERE incident_date IS NULL OR incident_date = ''
   OR incident_type IS NULL OR incident_type = '';

 * sqlite:///safety_data.db
Done.


employee_id,employee_name,department,incident_date,incident_type,days_lost,injury_severity
2,Jane Smith,Maintenance,,Fall,5,Moderate
4,Mary Johnson,Quality Control,2023-03-15,,2,Minor
10,Barbara Clark,Maintenance,2023-07-09,,2,Minor
17,David Wright,Quality Control,,Machine Accident,7,Severe
23,Matthew Perez,Manufacturing,,Slip,2,Minor


#### 4. Write a query to return the MIN, MAX and AVERAGE values of the days_lost column.

In [12]:
%%sql
UPDATE safety
SET days_lost = CASE
                   WHEN days_lost = '' THEN 1
                   ELSE days_lost
               END
WHERE days_lost = '';

 * sqlite:///safety_data.db
4 rows affected.


[]

In [13]:
%%sql
SELECT
    MIN(days_lost) AS min_value,
    MAX(days_lost) AS max_value,
    AVG(days_lost) AS avg_value
FROM safety;

 * sqlite:///safety_data.db
Done.


min_value,max_value,avg_value
1,10,4.233333333333333


#### 5. It has come to your attention that the missing values in the "incident_type" column is "fall" injuries. Update the table by replacing the missing values in the column with "fall" values

In [14]:
%%sql
UPDATE safety
SET incident_type = CASE
                        WHEN incident_type = '' THEN 'Fall'
                        ELSE incident_type
                    END
WHERE incident_type = '';

 * sqlite:///safety_data.db
2 rows affected.


[]

In [15]:
%%sql
SELECT COUNT(*) AS total_empty_string
FROM safety
WHERE incident_type = '';

 * sqlite:///safety_data.db
Done.


total_empty_string
0
